In [1]:
# filter some warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.colors as mcolors

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cmocean

from myfunctions import openpickle

In [ ]:
name = 'GFDL-CM4'
data_siconc = openpickle(name, '../../SO_data/data_siconc_w_area/')
data_sithick = openpickle(name, '../../SO_data/data_thick/')
data_mld = openpickle(name, '../../SO_data/data_mld/')
data_polynya = openpickle(name, '../../SO_data/data_polynya/')